# Basic Model Testing

In [2]:
import os
import numpy as np 
import torch
from torch import nn
from torch.nn import functional as F 
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt

In [3]:
#mod1 = torch.load('/datasets/home/27/827/ausant/ECE285 Project/MoDL_CenterNet/models/ctdet_coco_resdcn18.pth')

In [4]:
#for name in mod1['state_dict']:
#    print(name)

# Loading the Dataset

In [5]:
import os
import sys
sys.path.append(sys.path[0]+'/../lib') # Add library folder
#print(sys.path)

In [6]:
from opts import opts
from datasets.dataset_factory import get_dataset
from datasets.dataset.coco import COCO
from datasets.sample.ctdet import CTDetDataset

In [7]:
Dataset = get_dataset('coco', 'ctdet')

Create opt for passing to the constructor. \
Also pass a string with the training value

In [8]:
opt = type('', (), {})()
opt.data_dir = sys.path[0]+'/../../data/'
opt.task = 'ctdet'
split = 'train'

In [13]:
dataset = Dataset(opt,split)

==> initializing coco 2017 train data.
loading annotations into memory...
Done (t=14.16s)
creating index...
index created!
Loaded train 118287 samples


# Creating/Loading the Model

In [22]:
sys.path.append(sys.path[0]+'/../lib/models/networks/DCNv2')
from models.model import create_model, load_model, save_model
from trains.ctdet import CtdetTrainer

In [32]:
opt.lr = 1.25e-4 # default=1.25e-4
opt.num_classes = dataset.num_classes
opt.cat_spec_wh = True
opt.arch = 'res_18'
opt.heads = {'hm': opt.num_classes, 'wh': 2 if not opt.cat_spec_wh else 2 * opt.num_classes, 'reg': 2}
opt.head_conv = 64 # '64 for resnets and 256 for dla.'
opt.mse_loss = True
opt.reg_loss = True
opt.dense_wh = True # apply weighted regression near center or just apply regression on center point.
model = create_model(opt.arch, opt.heads, opt.head_conv)

=> loading pretrained model https://download.pytorch.org/models/resnet18-5c106cde.pth


In [33]:
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
trainer = CtdetTrainer(opt, model, optimizer)